In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from nba_api.stats.static import teams


In [3]:
nba_teams = teams.get_teams()

In [6]:
from nba_api.stats.endpoints import leaguegamefinder
gamefinder = leaguegamefinder.LeagueGameFinder()
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,72022,205,CCG,Celtics Crossover Gaming,2272200205,2022-07-27,CCG vs. WGS,None,36,22,...,NaN,1,3,4,6,0,0,2,2,0.0
1,72022,229,WGS,Warriors Gaming Squad,2272200205,2022-07-27,WGS @ CCG,None,36,15,...,NaN,1,1,2,3,1,0,0,4,0.0
2,72022,205,CCG,Celtics Crossover Gaming,2272200204,2022-07-27,CCG @ WGS,None,36,17,...,NaN,1,1,2,3,1,0,3,1,-2.4
3,72022,229,WGS,Warriors Gaming Squad,2272200204,2022-07-27,WGS vs. CCG,None,36,21,...,NaN,2,1,3,6,3,0,1,1,2.4
4,72022,215,LKG,Lakers Gaming,2272200208,2022-07-27,LKG vs. SGC,None,36,17,...,NaN,1,2,3,3,0,0,2,3,-3.0


In [30]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          30000 non-null  object 
 1   TEAM_ID            30000 non-null  int64  
 2   TEAM_ABBREVIATION  30000 non-null  object 
 3   TEAM_NAME          30000 non-null  object 
 4   GAME_ID            30000 non-null  object 
 5   GAME_DATE          30000 non-null  object 
 6   MATCHUP            30000 non-null  object 
 7   WL                 28474 non-null  object 
 8   MIN                30000 non-null  int64  
 9   PTS                30000 non-null  int64  
 10  FGM                30000 non-null  int64  
 11  FGA                30000 non-null  int64  
 12  FG_PCT             29993 non-null  float64
 13  FG3M               30000 non-null  int64  
 14  FG3A               30000 non-null  int64  
 15  FG3_PCT            29983 non-null  float64
 16  FTM                300

In [27]:
# Subset the games to when the last 4 digits of SEASON_ID were 2021.
games_2021 = games[games.SEASON_ID == '42021']
games_2021

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1505,42021,1610612744,GSW,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,241,103,...,1.000,15,29,44,27,13,7,15,20,13.0
1506,42021,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,239,90,...,0.917,11,30,41,27,8,8,22,16,-13.0
1705,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.867,4,35,39,23,9,2,6,28,10.0
1706,42021,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,238,94,...,0.677,8,39,47,18,2,2,18,16,-10.0
1747,42021,1610612738,BOS,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,241,97,...,0.737,11,31,42,22,8,6,15,17,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,42021,212,JZG,Jazz Gaming,1242100102,2021-08-26,JZG @ BZG,W,240,84,...,0.625,5,11,16,23,8,3,6,16,3.0
7401,42021,200,SGC,76ers GC,1242100131,2021-08-26,SGC @ NGC,L,240,62,...,0.667,7,10,17,16,3,1,4,12,-10.0
7402,42021,200,SGC,76ers GC,1242100133,2021-08-26,SGC @ NGC,L,240,69,...,NaN,7,10,17,11,3,3,5,13,-11.0
7403,42021,214,KNX,Knicks Gaming,1242100113,2021-08-26,KNX @ GZG,W,240,76,...,0.750,5,17,22,12,8,9,6,10,21.0


In [25]:
games_2022 = games[games.SEASON_ID.str[-4:] == '2022']
games_2022

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,72022,205,CCG,Celtics Crossover Gaming,2272200205,2022-07-27,CCG vs. WGS,None,36,22,...,NaN,1,3,4,6,0,0,2,2,0.0
1,72022,229,WGS,Warriors Gaming Squad,2272200205,2022-07-27,WGS @ CCG,None,36,15,...,NaN,1,1,2,3,1,0,0,4,0.0
2,72022,205,CCG,Celtics Crossover Gaming,2272200204,2022-07-27,CCG @ WGS,None,36,17,...,NaN,1,1,2,3,1,0,3,1,-2.4
3,72022,229,WGS,Warriors Gaming Squad,2272200204,2022-07-27,WGS vs. CCG,None,36,21,...,NaN,2,1,3,6,3,0,1,1,2.4
4,72022,215,LKG,Lakers Gaming,2272200208,2022-07-27,LKG vs. SGC,None,36,17,...,NaN,1,2,3,3,0,0,2,3,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340,12022,241,DUX,DUX Infinitos,1212200001,2022-04-05,DUX @ PCG,W,240,70,...,1.0,6,10,16,13,4,1,7,15,-3.0
3341,12022,217,MVG,Mavs Gaming,1212200002,2022-04-05,MVG vs. BCG,L,240,48,...,1.0,6,12,18,11,6,1,11,9,-8.0
3343,12022,201,BCG,Bucks Gaming,1212200002,2022-04-05,BCG @ MVG,W,240,61,...,1.0,10,10,20,6,6,1,6,11,8.0
3346,12022,202,WDG,Wizards District Gaming,1212200003,2022-04-05,WDG @ GEN,L,240,43,...,1.0,10,10,20,10,3,4,7,14,-21.0


In [12]:
games_df = pd.concat([games_2021, games_2022], axis=0)
games_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1505,42021,1610612744,GSW,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,241,103,...,1.000,15,29,44,27,13,7,15,20,13.0
1506,42021,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,239,90,...,0.917,11,30,41,27,8,8,22,16,-13.0
1705,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.867,4,35,39,23,9,2,6,28,10.0
1706,42021,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,238,94,...,0.677,8,39,47,18,2,2,18,16,-10.0
1747,42021,1610612738,BOS,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,241,97,...,0.737,11,31,42,22,8,6,15,17,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3340,12022,241,DUX,DUX Infinitos,1212200001,2022-04-05,DUX @ PCG,W,240,70,...,1.000,6,10,16,13,4,1,7,15,-3.0
3341,12022,217,MVG,Mavs Gaming,1212200002,2022-04-05,MVG vs. BCG,L,240,48,...,1.000,6,12,18,11,6,1,11,9,-8.0
3343,12022,201,BCG,Bucks Gaming,1212200002,2022-04-05,BCG @ MVG,W,240,61,...,1.000,10,10,20,6,6,1,6,11,8.0
3346,12022,202,WDG,Wizards District Gaming,1212200003,2022-04-05,WDG @ GEN,L,240,43,...,1.000,10,10,20,10,3,4,7,14,-21.0


In [14]:
winning_df = games_df[games_df.WL == 'W']
winning_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1505,42021,1610612744,GSW,Golden State Warriors,0042100406,2022-06-16,GSW @ BOS,W,241,103,...,1.000,15,29,44,27,13,7,15,20,13.0
1705,42021,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,W,240,104,...,0.867,4,35,39,23,9,2,6,28,10.0
1748,42021,1610612744,GSW,Golden State Warriors,0042100404,2022-06-10,GSW @ BOS,W,241,107,...,0.800,16,39,55,20,10,5,16,21,10.0
1780,42021,1610612738,BOS,Boston Celtics,0042100403,2022-06-08,BOS vs. GSW,W,240,116,...,0.708,15,32,47,28,7,7,12,17,16.0
1790,42021,1610612744,GSW,Golden State Warriors,0042100402,2022-06-05,GSW vs. BOS,W,241,107,...,0.700,6,36,42,25,15,2,12,17,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,12022,229,WGS,Warriors Gaming Squad,1212200005,2022-04-05,WGS @ KGG,W,240,68,...,1.000,8,12,20,11,5,4,6,10,10.0
3331,12022,205,CCG,Celtics Crossover Gaming,1212200004,2022-04-05,CCG vs. HTG,W,240,80,...,1.000,5,11,16,24,4,3,1,12,29.0
3339,12022,240,GEN,Gen.G Tigers,1212200003,2022-04-05,GEN vs. WDG,W,240,63,...,1.000,8,13,21,15,7,1,6,11,21.0
3340,12022,241,DUX,DUX Infinitos,1212200001,2022-04-05,DUX @ PCG,W,240,70,...,1.000,6,10,16,13,4,1,7,15,-3.0


In [16]:
lost_df = games_df[games_df.WL == 'L']
lost_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1506,42021,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,L,239,90,...,0.917,11,30,41,27,8,8,22,16,-13.0
1706,42021,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,L,238,94,...,0.677,8,39,47,18,2,2,18,16,-10.0
1747,42021,1610612738,BOS,Boston Celtics,0042100404,2022-06-10,BOS vs. GSW,L,241,97,...,0.737,11,31,42,22,8,6,15,17,-10.0
1781,42021,1610612744,GSW,Golden State Warriors,0042100403,2022-06-08,GSW @ BOS,L,237,100,...,0.867,6,25,31,22,7,5,16,23,-16.0
1789,42021,1610612738,BOS,Boston Celtics,0042100402,2022-06-05,BOS @ GSW,L,242,88,...,0.765,6,37,43,24,5,7,18,18,-19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,12022,220,PCG,Pacers Gaming,1212200001,2022-04-05,PCG vs. DUX,L,240,68,...,1.000,10,8,18,23,6,1,7,11,3.0
3338,12022,213,KGG,Kings Guard Gaming,1212200005,2022-04-05,KGG vs. WGS,L,240,54,...,NaN,9,8,17,17,4,4,7,9,-10.0
3341,12022,217,MVG,Mavs Gaming,1212200002,2022-04-05,MVG vs. BCG,L,240,48,...,1.000,6,12,18,11,6,1,11,9,-8.0
3346,12022,202,WDG,Wizards District Gaming,1212200003,2022-04-05,WDG @ GEN,L,240,43,...,1.000,10,10,20,10,3,4,7,14,-21.0


________________________________________________

## Exploring Winning Games/Teams dataframe:

In [19]:
#exploring winning df :
winning_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3651 entries, 1505 to 3343
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          3651 non-null   object 
 1   TEAM_ID            3651 non-null   int64  
 2   TEAM_ABBREVIATION  3651 non-null   object 
 3   TEAM_NAME          3651 non-null   object 
 4   GAME_ID            3651 non-null   object 
 5   GAME_DATE          3651 non-null   object 
 6   MATCHUP            3651 non-null   object 
 7   WL                 3651 non-null   object 
 8   MIN                3651 non-null   int64  
 9   PTS                3651 non-null   int64  
 10  FGM                3651 non-null   int64  
 11  FGA                3651 non-null   int64  
 12  FG_PCT             3651 non-null   float64
 13  FG3M               3651 non-null   int64  
 14  FG3A               3651 non-null   int64  
 15  FG3_PCT            3651 non-null   float64
 16  FTM                36

In [21]:
#what is the mean/average of winning teams 3pt made?
winning_df.FG3M.mean() 

11.666118871542043

In [22]:
#what is the mean/average of winning teams 3pt %%?
winning_df.FG3_PCT.mean() 

0.4259528896192813

In [23]:
#what is the mean/average of winning teams 3pt attempts?
winning_df.FG3A.mean() 

28.88195015064366

__________________________